SPM12 tutorial using Octave

This tutorial will show how to preprocess fMRI data using the SPM12 toolbox

This Jupyter notebook utilizes an Octave kernel on Binder, offering a cloud-based platform for running SPM12 and Matlab scripts. It creates a fully reproducible environment for fMRI analysis, ensuring that anyone with internet access can run the same code on the same dataset and achieve identical results. In addition to being a powerful resource for tutorials and collaboration, it also serves as a means to openly share the data and methods that underpin scientific research


In [ ]:
% Set the subject and SPM12 directories
data_dir = fullfile(getenv('PWD'),'sub-01');
spm_dir = fullfile(getenv('PWD'),'spm12');

In [ ]:
# Create the prep, stat and results folders
data_res = [data_dir '/results'];
mkdir(data_res);
data_prep = [data_dir '/prep'];
mkdir(data_prep);
data_stat = [data_dir '/stat'];
mkdir(data_stat);

In [ ]:
% Set the processing directory
processing_dir = data_prep;

In [ ]:
% Load the zipped and unzipped structural data
#s_raw_fn = .zip whereas s_fn = .nii
s_raw_fn = fullfile(data_dir, 'anat', ['sub-01' '_T1w.nii.gz']);
if exist(s_raw_fn, 'file')
    out_fns = gunzip(s_raw_fn);
    s_fn = out_fns{1};
else
    s_fn = strrep(s_raw_fn, '.gz', '');
end

In [ ]:
% Load the zipped and unzipped functional data
f_raw_fn = fullfile(data_dir, 'func', ['sub-01' '_task-ft_run-1_bold.nii.gz']);
if exist(f_raw_fn, 'file')
    out_fns = gunzip(f_raw_fn);
    f_fn = out_fns{1};
else
    f_fn = strrep(f_raw_fn, '.gz', '');
end

In [ ]:
% Create preprocessing subfolders anat and func
anat_dir = fullfile(processing_dir, 'anat'); 
func_dir = fullfile(processing_dir, 'func'); 
if ~exist(anat_dir, 'dir')
    mkdir(anat_dir)
end
if ~exist(func_dir, 'dir')
    mkdir(func_dir)
end

In [ ]:
% Copy extracted files to the corresponding preprocessing subfolders
anat_fn = fullfile(processing_dir, 'anat', ['sub-01' '_T1w.nii']);
func_fn = fullfile(processing_dir, 'func', ['sub-01' '_task-ft_run-1_bold.nii']);
if ~exist(anat_fn, 'file')
    copyfile(s_fn, anat_dir)
end
if ~exist(func_fn, 'file')
    copyfile(f_fn, func_dir)
end

In [ ]:
addpath (fullfile (getenv('PWD'), 'spm12')); savepath ();
addpath (fullfile (getenv('PWD'), 'dicm2nii-0.2')); savepath ();

Now it is time to show the structural data using the function fmrwhy_util_readNifti

In [ ]:
graphics_toolkit ("gnuplot");
[p_func, frm1, rg1, dim1] = fmrwhy_util_readNifti(anat_fn);
struct_4Dimg = p_func.nii.img;
[Ni, Nj, Nk, Nt] = size(struct_4Dimg);
subplot(131); imagesc(rot90(squeeze(struct_4Dimg(round(Ni/2),:,:,1))));
subplot(132); imagesc(rot90(squeeze(struct_4Dimg(:,round(Nj/2),:,1))));
subplot(133); imagesc(rot90(squeeze(struct_4Dimg(:,:,round(Nk/2),1))));

In [ ]:
% TO-DO
% Question: what did we plot in the previous cell?
% Try to plot another specific slice of the Image (hint: check the size of the structural image first)

Here we start the pre-processing of the fMRI data

The preprocessing pipeline in SPM12 prepares raw fMRI data for statistical analysis by correcting for various artifacts and aligning the images across time and space. 

The key preprocessing steps include:

1. Slice Timing Correction (STC)
Corrects for differences in slice acquisition times due to sequential or interleaved scanning.
Necessary for event-related fMRI but less critical for block designs.

3. Realignment
Corrects for head motion by aligning all fMRI volumes to a reference volume (usually the first or mean image).
Produces motion parameters that can be used as nuisance regressors in later analyses.


5. Co-registration
Aligns the functional (EPI) images with a high-resolution anatomical image (T1-weighted).
Improves spatial accuracy in normalization and subsequent analyses.


7. Segmentation and Normalization
Segments the T1-weighted anatomical image into different tissue types (gray matter, white matter, CSF).
Computes spatial transformations to align the subject's brain to a standard template (e.g., MNI space).
Applies the transformation to both anatomical and functional images.


9. Smoothing
Applies a Gaussian kernel (e.g., 6-8 mm FWHM) to the normalized functional images.
Improves signal-to-noise ratio and allows for better inter-subject comparisons.


11. (Optional) Denoising and Artifact Correction
Physiological noise correction (e.g., CompCor, RETROICOR, or ICA-based denoising).
Motion scrubbing (framewise displacement-based exclusion).
Nuisance regression (e.g., removing motion parameters, CSF, and white matter signals).

In [ ]:
%  Realignment
f4D_spm = spm_vol(func_fn);
spm_size = size(f4D_spm);
Nt = spm_size(1);
% Declare output structure used during the whole analysis
output = struct;

In [ ]:
% STEP 1 -- Realign (estimate and reslice) all functionals to first functional
disp('Realign all volumes to first functional volume');
spm('defaults','fmri');
spm_jobman('initcfg');
realign_estimate_reslice = struct;
% Data
fnms={};
for i = 1:Nt
    fnms{i} = [func_fn ',' num2str(i) ];
end
realign_estimate_reslice.matlabbatch{1}.spm.spatial.realign.estwrite.data={fnms'}; # This is a spatial transformation

In [ ]:
% Estimate Options
realign_estimate_reslice.matlabbatch{1}.spm.spatial.realign.estwrite.eoptions.quality = 0.9;
realign_estimate_reslice.matlabbatch{1}.spm.spatial.realign.estwrite.eoptions.sep = 4;
realign_estimate_reslice.matlabbatch{1}.spm.spatial.realign.estwrite.eoptions.fwhm = 5;
realign_estimate_reslice.matlabbatch{1}.spm.spatial.realign.estwrite.eoptions.rtm = 1;
realign_estimate_reslice.matlabbatch{1}.spm.spatial.realign.estwrite.eoptions.interp = 2;
realign_estimate_reslice.matlabbatch{1}.spm.spatial.realign.estwrite.eoptions.wrap = [0 0 0];
realign_estimate_reslice.matlabbatch{1}.spm.spatial.realign.estwrite.eoptions.weight = '';

In [ ]:
% Reslice Options
realign_estimate_reslice.matlabbatch{1}.spm.spatial.realign.estwrite.roptions.which = [2 1];
realign_estimate_reslice.matlabbatch{1}.spm.spatial.realign.estwrite.roptions.interp = 4;
realign_estimate_reslice.matlabbatch{1}.spm.spatial.realign.estwrite.roptions.wrap = [0 0 0];
realign_estimate_reslice.matlabbatch{1}.spm.spatial.realign.estwrite.roptions.mask = 1;
realign_estimate_reslice.matlabbatch{1}.spm.spatial.realign.estwrite.roptions.prefix = 'r';

In [ ]:
% Run the job
spm_jobman('run',realign_estimate_reslice.matlabbatch);

In [ ]:
[d, f, e] = fileparts(func_fn);
output.rfunctional_fn = [d filesep 'r' f e]; % Here we define the appendix for the realign corrected file (new)
output.mp_fn = [d filesep 'rp_' f '.txt']; % This is the file containing the Rigid Body transformations (Tx, Ty, Tz, R, P, Y)
output.RB = load(output.mp_fn);
output.mean_functional_fn = [d filesep 'mean' f e];

disp('Realignment - Done!');

In [ ]:
% TO-DO
% Check if you can find the newly generated file and identify its name

In [ ]:
% Now we will perform the STC
spm('defaults','fmri');
spm_jobman('initcfg');

stc_estimate = struct;

% Data
fnms={};
for i = 1:Nt
    fnms{i} = [output.rfunctional_fn  ',' num2str(i) ];
end
stc_estimate. matlabbatch{1}.spm.temporal.st.scans = {fnms'}; # Now it is temporal

In [ ]:
% Here some parameters that you need to change according to your data acquisition
stc_estimate.matlabbatch{1}.spm.temporal.st.nslices = 34;
stc_estimate.matlabbatch{1}.spm.temporal.st.tr = 2;
stc_estimate.matlabbatch{1}.spm.temporal.st.ta = 2-2/34;
stc_estimate.matlabbatch{1}.spm.temporal.st.so = [2:2:34 1:2:34]; # Interleaved mode
stc_estimate.matlabbatch{1}.spm.temporal.st.refslice = 34;

In [ ]:
% Run the job
spm_jobman('run', stc_estimate.matlabbatch);

In [ ]:
[d, f, e] = fileparts(output.rfunctional_fn);
output.stc_unctional_fn = [d filesep 'a' f e]; % Here we define the appendix for the STC (realigned) corrected file (new)

disp('STC - Done!');

In [ ]:
% TO-DO
% Check if you can find the newly generated file and identify its name

In [ ]:
% Now we will coregister the structural image to first functional image (estimate only)

disp('Coregister structural image to first dynamic image');
spm('defaults','fmri');
spm_jobman('initcfg');
coreg_estimate = struct;

% Reference image
coreg_estimate.matlabbatch{1}.spm.spatial.coreg.estimate.ref = {output.mean_functional_fn};
% Source image
coreg_estimate.matlabbatch{1}.spm.spatial.coreg.estimate.source = {anat_fn};

% Here we use the mean functional as a reference and we co-register the structural accordingly

In [ ]:
% Estimate Options
coreg_estimate.matlabbatch{1}.spm.spatial.coreg.estimate.eoptions.cost_fun = 'nmi';
coreg_estimate.matlabbatch{1}.spm.spatial.coreg.estimate.eoptions.sep = [4 2];
coreg_estimate.matlabbatch{1}.spm.spatial.coreg.estimate.eoptions.tol = [0.02 0.02 0.02 0.001 0.001 0.001 0.01 0.01 0.01 0.001 0.001 0.001];
coreg_estimate.matlabbatch{1}.spm.spatial.coreg.estimate.eoptions.fwhm = [7 7];

In [ ]:
% Run the job
spm_jobman('run',coreg_estimate.matlabbatch);

disp('CoRegistration Done!');

In [ ]:
% TO-DO
% Did you generate a new file?
% Explain your answer

In [ ]:
% Segmentation of coregistered structural image into GM, WM, CSF, etc
% (with implicit warping to MNI space, saving forward and inverse transformations)
disp('Segmentation');
spm('defaults','fmri');
spm_jobman('initcfg');
segmentation = struct;
spm_dir= 'spm12'

In [ ]:
% Parameters
% Channel
segmentation.matlabbatch{1}.spm.spatial.preproc.channel.biasreg = 0.001;
segmentation.matlabbatch{1}.spm.spatial.preproc.channel.biasfwhm = 60;
segmentation.matlabbatch{1}.spm.spatial.preproc.channel.write = [0 1];
segmentation.matlabbatch{1}.spm.spatial.preproc.channel.vols = {anat_fn}; % Here the structural input
% Tissue
for t = 1:6
    segmentation.matlabbatch{1}.spm.spatial.preproc.tissue(t).tpm = {[spm_dir filesep 'tpm' filesep 'TPM.nii,' num2str(t)]};
    segmentation.matlabbatch{1}.spm.spatial.preproc.tissue(t).ngaus = t-1;
    segmentation.matlabbatch{1}.spm.spatial.preproc.tissue(t).native = [1 0];
    segmentation.matlabbatch{1}.spm.spatial.preproc.tissue(t).warped = [0 0];
end
segmentation.matlabbatch{1}.spm.spatial.preproc.tissue(1).ngaus = 1;
segmentation.matlabbatch{1}.spm.spatial.preproc.tissue(6).ngaus = 2;
% Warp
segmentation.matlabbatch{1}.spm.spatial.preproc.warp.mrf = 1;
segmentation.matlabbatch{1}.spm.spatial.preproc.warp.cleanup = 1;
segmentation.matlabbatch{1}.spm.spatial.preproc.warp.reg = [0 0.001 0.5 0.05 0.2];
segmentation.matlabbatch{1}.spm.spatial.preproc.warp.affreg = 'mni';
segmentation.matlabbatch{1}.spm.spatial.preproc.warp.fwhm = 0;
segmentation.matlabbatch{1}.spm.spatial.preproc.warp.samp = 3;
segmentation.matlabbatch{1}.spm.spatial.preproc.warp.write=[1 1];

In [ ]:
% Run
spm_jobman('run',segmentation.matlabbatch);

In [ ]:
[d, f, e] = fileparts(anat_fn);
output.forward_transformation = [d filesep 'y_' f e];
output.inverse_transformation = [d filesep 'iy_' f e];
output.biascorrected_structural = [d filesep 'm' f e];;
output.gm_fn = [d filesep 'c1' f e];
output.wm_fn = [d filesep 'c2' f e];
output.csf_fn = [d filesep 'c3' f e];
output.bone_fn = [d filesep 'c4' f e];
output.soft_fn = [d filesep 'c5' f e];
output.air_fn = [d filesep 'c6' f e];
disp('Segmentation - done!');

In [ ]:
disp('Normalize the functional data');
spm('defaults','fmri');
spm_jobman('initcfg');
normalize = struct;
% Data
fns={};
for i = 1:Nt
    fns{i} = [output.stc_unctional_fn ',' num2str(i) ]; %Why this data?
end

In [ ]:
% Parameters
normalize.matlabbatch{1}.spm.spatial.normalise.write.subj.def = {output.forward_transformation};
normalize.matlabbatch{1}.spm.spatial.normalise.write.subj.resample= fns';
normalize.matlabbatch{1}.spm.spatial.normalise.write.woptions.bb = [-78 -112 -70;78 76 85];
normalize.matlabbatch{1}.spm.spatial.normalise.write.woptions.vox = [3 3 3];
normalize.matlabbatch{1}.spm.spatial.normalise.write.woptions.interp = 4;
normalize.matlabbatch{1}.spm.spatial.normalise.write.woptions.prefix = 'w';

In [ ]:
% Run
spm_jobman('run',normalize.matlabbatch);

In [ ]:
[d, f, e] = fileparts(output.stc_unctional_fn);
output.normalized_fn = [d filesep 'w' f e];
disp('Normalization of the functional data - done!');

In [ ]:
disp('Normalize the structural data');
spm('defaults','fmri');
spm_jobman('initcfg');
normalize = struct;

% Parameters
normalize.matlabbatch{1}.spm.spatial.normalise.write.subj.def = {output.forward_transformation};
normalize.matlabbatch{1}.spm.spatial.normalise.write.subj.resample= {output.biascorrected_structural};
normalize.matlabbatch{1}.spm.spatial.normalise.write.woptions.bb = [-78 -112 -70;78 76 85];
normalize.matlabbatch{1}.spm.spatial.normalise.write.woptions.vox = [1 1 1]; % Why this difference with functional data?
normalize.matlabbatch{1}.spm.spatial.normalise.write.woptions.interp = 4;
normalize.matlabbatch{1}.spm.spatial.normalise.write.woptions.prefix = 'w'; % New prefix

In [ ]:
% TO-DO
% Describe why this difference

In [ ]:
% Run
spm_jobman('run',normalize.matlabbatch);

In [ ]:
[d, f, e] = fileparts(output.biascorrected_structural);
output.normalized_struct = [d filesep 'w' f e];
disp('Normalization of the structural data - done!');

In [ ]:
% Now we will apply a Gaussian kernel smoothing on realigned, slice time corrected, normalized functional data
disp('Gaussian kernel smoothing of functional data');
spm('defaults','fmri');
spm_jobman('initcfg');
smooth = struct;
% Data
fns={};
for i = 1:Nt
    fns{i} = [output.normalized_fn ',' num2str(i) ];
end

smooth.matlabbatch{1}.spm.spatial.smooth.data = fns';




In [ ]:
% Parameters
smooth.matlabbatch{1}.spm.spatial.smooth.fwhm = [8 8 8]; % Why 8 mm?
smooth.matlabbatch{1}.spm.spatial.smooth.dtype = 0;
smooth.matlabbatch{1}.spm.spatial.smooth.im = 0;
smooth.matlabbatch{1}.spm.spatial.smooth.prefix = 's'; % New prefix

In [ ]:
% TO-DO
% Describe why 8 mm?

In [ ]:
% Run the job
spm_jobman('run',smooth.matlabbatch);

In [ ]:
[d, f, e] = fileparts(output.normalized_fn);
output.srfunctional_fn = [d filesep 's' f e]; % This data will serve as input for your model (design matrix)
disp('Smoothing - done!');

In [ ]:
% Define the Design Matrix with its regressors: R, L, B and combination of them
spm('defaults','fmri');
spm_jobman('initcfg');
design_stats = struct;
func4D_spm = spm_vol(output.srfunctional_fn);
func4D_size = size(func4D_spm);
swarfs = spm_select('expand', fullfile(output.srfunctional_fn));

% SETUP BATCH JOB STRUCTURE
% dir
design_stats.matlabbatch{1}.spm.stats.fmri_spec.dir = {data_stat}; 
% timing
design_stats.matlabbatch{1}.spm.stats.fmri_spec.timing.units = 'secs';
design_stats.matlabbatch{1}.spm.stats.fmri_spec.timing.RT = 2;
design_stats.matlabbatch{1}.spm.stats.fmri_spec.timing.fmri_t = 16;
design_stats.matlabbatch{1}.spm.stats.fmri_spec.timing.fmri_t0 = 8;
% sess
design_stats.matlabbatch{1}.spm.stats.fmri_spec.sess.scans = cellstr(swarfs);




design_stats.matlabbatch{1}.spm.stats.fmri_spec.sess.cond(1).name = 'R';
design_stats.matlabbatch{1}.spm.stats.fmri_spec.sess.cond(1).onset = [20
                                                         100
                                                         180
                                                         260
                                                         340];
design_stats.matlabbatch{1}.spm.stats.fmri_spec.sess.cond(1).duration = 20;
design_stats.matlabbatch{1}.spm.stats.fmri_spec.sess.cond(1).tmod = 0;
design_stats.matlabbatch{1}.spm.stats.fmri_spec.sess.cond(1).pmod = struct('name', {}, 'param', {}, 'poly', {});
design_stats.matlabbatch{1}.spm.stats.fmri_spec.sess.cond(1).orth = 1;

design_stats.matlabbatch{1}.spm.stats.fmri_spec.sess.cond(2).name = 'L';
design_stats.matlabbatch{1}.spm.stats.fmri_spec.sess.cond(2).onset = [60
                                                         140
                                                         220
                                                         300
                                                         380];
design_stats.matlabbatch{1}.spm.stats.fmri_spec.sess.cond(2).duration = 20;
design_stats.matlabbatch{1}.spm.stats.fmri_spec.sess.cond(2).tmod = 0;
design_stats.matlabbatch{1}.spm.stats.fmri_spec.sess.cond(2).pmod = struct('name', {}, 'param', {}, 'poly', {});
design_stats.matlabbatch{1}.spm.stats.fmri_spec.sess.cond(2).orth = 1;

design_stats.matlabbatch{1}.spm.stats.fmri_spec.sess.cond(3).name = 'B';
design_stats.matlabbatch{1}.spm.stats.fmri_spec.sess.cond(3).onset = [0
                                                         40
                                                         80
                                                         120
                                                         160
                                                         200
                                                         240
                                                         280
                                                         320
                                                         360
                                                         400];
%%
design_stats.matlabbatch{1}.spm.stats.fmri_spec.sess.cond(3).duration = 20;
design_stats.matlabbatch{1}.spm.stats.fmri_spec.sess.cond(3).tmod = 0;
design_stats.matlabbatch{1}.spm.stats.fmri_spec.sess.cond(3).pmod = struct('name', {}, 'param', {}, 'poly', {});
design_stats.matlabbatch{1}.spm.stats.fmri_spec.sess.cond(3).orth = 1;
design_stats.matlabbatch{1}.spm.stats.fmri_spec.sess.multi = {''};
design_stats.matlabbatch{1}.spm.stats.fmri_spec.sess.regress = struct('name', {}, 'val', {});
design_stats.matlabbatch{1}.spm.stats.fmri_spec.sess.multi_reg = {output.mp_fn};
design_stats.matlabbatch{1}.spm.stats.fmri_spec.sess.hpf = 128;
design_stats.matlabbatch{1}.spm.stats.fmri_spec.fact = struct('name', {}, 'levels', {});
design_stats.matlabbatch{1}.spm.stats.fmri_spec.bases.hrf.derivs = [0 0];
design_stats.matlabbatch{1}.spm.stats.fmri_spec.volt = 1;
design_stats.matlabbatch{1}.spm.stats.fmri_spec.global = 'None';
design_stats.matlabbatch{1}.spm.stats.fmri_spec.mthresh = 0.8;
design_stats.matlabbatch{1}.spm.stats.fmri_spec.mask = {''};
design_stats.matlabbatch{1}.spm.stats.fmri_spec.cvi = 'AR(1)';

In [ ]:
% RUN BATCH JOB
spm_jobman('run',design_stats.matlabbatch);

In [ ]:
% Estimate the Betas
spm('defaults','fmri');
spm_jobman('initcfg');
model_estimation = struct;
% spmmat
model_estimation.matlabbatch{1}.spm.stats.fmri_est.spmmat = {[data_stat filesep 'SPM.mat']};
% write_residuals
model_estimation.matlabbatch{1}.spm.stats.fmri_est.write_residuals = 0;
% method
model_estimation.matlabbatch{1}.spm.stats.fmri_est.method.Classical = 1;
% RUN BATCH JOB
spm_jobman('run',model_estimation.matlabbatch);

In [ ]:
% Estimate the contrasts and SPMs
spm('defaults','fmri');
spm_jobman('initcfg');

contrast = struct;
% spmmat

contrast.matlabbatch{1}.spm.stats.con.spmmat = {[data_stat filesep 'SPM.mat']};

%contrast
contrast.matlabbatch{1}.spm.stats.con.consess{1}.tcon.name = 'R';
contrast.matlabbatch{1}.spm.stats.con.consess{1}.tcon.weights = 1;
contrast.matlabbatch{1}.spm.stats.con.consess{1}.tcon.sessrep = 'none';
contrast.matlabbatch{1}.spm.stats.con.consess{2}.tcon.name = 'L';
contrast.matlabbatch{1}.spm.stats.con.consess{2}.tcon.weights = [0 1];
contrast.matlabbatch{1}.spm.stats.con.consess{2}.tcon.sessrep = 'none';
contrast.matlabbatch{1}.spm.stats.con.consess{3}.tcon.name = 'RvsL';
contrast.matlabbatch{1}.spm.stats.con.consess{3}.tcon.weights = [1 -1];
contrast.matlabbatch{1}.spm.stats.con.consess{3}.tcon.sessrep = 'none';
contrast.matlabbatch{1}.spm.stats.con.delete = 0;
% RUN BATCH JOB
spm_jobman('run',contrast.matlabbatch);

In [ ]:
% Show the results
spm('defaults','fmri');
spm_jobman('initcfg');
% SETUP BATCH JOB STRUCTURE
results = struct;
% spmmat
results.matlabbatch{1}.spm.stats.results.spmmat = {[data_stat filesep 'SPM.mat']};
% conspec
results.matlabbatch{1}.spm.stats.results.conspec.titlestr = '';
results.matlabbatch{1}.spm.stats.results.conspec.contrasts = 3;
results.matlabbatch{1}.spm.stats.results.conspec.threshdesc = 'FWE';
results.matlabbatch{1}.spm.stats.results.conspec.thresh = 0.0500;
results.matlabbatch{1}.spm.stats.results.conspec.extent = 0;
results.matlabbatch{1}.spm.stats.results.conspec.conjunction = 1;
results.matlabbatch{1}.spm.stats.results.conspec.mask.none = 1;
% units
results.matlabbatch{1}.spm.stats.results.units = 1;
% export
results.matlabbatch{1}.spm.stats.results.export{1}.ps = 1;
% RUN BATCH JOB
spm_jobman('run',results.matlabbatch);